### 데이터 분석 및 지도 시각화 Library Load

In [1]:
import pandas as pd
import json
import folium

### 데이터 전처리

In [2]:
df_1 = pd.read_csv("./전력량_주택_마커.csv")
df_1.drop(columns=['Unnamed: 0'], inplace=True)
df_1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448 entries, 0 to 2447
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   연도       2448 non-null   int64  
 1   월        2448 non-null   int64  
 2   시도       2448 non-null   object 
 3   주택가격변동률  2448 non-null   float64
 4   사용량      2448 non-null   int64  
 5   위도       2448 non-null   float64
 6   경도       2448 non-null   float64
dtypes: float64(3), int64(3), object(1)
memory usage: 134.0+ KB


In [7]:
df_1.head()

,연도,월,시도,주택가격변동률,사용량,위도,경도
0,2010,1,서울특별시,0.13,4398183,37.551638,126.993243
1,2010,2,서울특별시,0.23,4168168,37.551638,126.993243
2,2010,3,서울특별시,0.07,3624186,37.551638,126.993243
3,2010,4,서울특별시,-0.02,3719082,37.551638,126.993243
4,2010,5,서울특별시,-0.25,3426951,37.551638,126.993243


In [5]:
year = 2010
month = 1

df_ver = df_1.query('연도 == @year & 월 == @month')

In [6]:
df_ver

,연도,월,시도,주택가격변동률,사용량,위도,경도
0,2010,1,서울특별시,0.13,4398183,37.551638,126.993243
144,2010,1,부산광역시,0.36,1848492,35.209853,129.069121
288,2010,1,대구광역시,0.09,1331473,35.825757,128.576897
432,2010,1,인천광역시,0.05,2016429,37.455307,126.703920
576,2010,1,광주광역시,0.07,712924,35.154840,126.836647
720,2010,1,대전광역시,0.63,826373,36.339403,127.396923
864,2010,1,울산광역시,0.24,2259608,35.553064,129.254432
1008,2010,1,세종특별자치시,0.00,0,36.560326,127.258638
1152,2010,1,경기도,-0.06,8725482,37.412505,127.520017
1296,2010,1,강원도,0.13,1480180,37.820812,128.156579


In [6]:
df_data = df_ver[['시도', '사용량']]
df_data

,시도,사용량
0,강원도,1480180
1,경기도,8725482
2,경상남도,2922775
3,경상북도,3743904
4,광주광역시,712924
5,대구광역시,1331473
6,대전광역시,826373
7,부산광역시,1848492
8,서울특별시,4398183
9,세종특별자치시,0


In [7]:
df_data_test = df_data.to_dict(orient='records')
df_data_test

[{'시도': '강원도', '사용량': 1480180},
 {'시도': '경기도', '사용량': 8725482},
 {'시도': '경상남도', '사용량': 2922775},
 {'시도': '경상북도', '사용량': 3743904},
 {'시도': '광주광역시', '사용량': 712924},
 {'시도': '대구광역시', '사용량': 1331473},
 {'시도': '대전광역시', '사용량': 826373},
 {'시도': '부산광역시', '사용량': 1848492},
 {'시도': '서울특별시', '사용량': 4398183},
 {'시도': '세종특별자치시', '사용량': 0},
 {'시도': '울산광역시', '사용량': 2259608},
 {'시도': '인천광역시', '사용량': 2016429},
 {'시도': '전라남도', '사용량': 2233687},
 {'시도': '전라북도', '사용량': 1707366},
 {'시도': '제주특별자치도', '사용량': 325800},
 {'시도': '충청남도', '사용량': 3408166},
 {'시도': '충청북도', '사용량': 1755905}]

In [8]:
jsonfile = open('Sido.json','r',encoding='utf8').read()     # 시도구분된 지역 json 파일 가져오기
jsondata = json.loads(jsonfile)                             
jsondata_loc = {'type': 'FeatureCollection'}                # Sido.json을 복사해서 일부 특성(Key와 Record) 추가
jsondata_pick = []
for idx in jsondata['features']:
    idx['id'] = idx['properties']['CTP_KOR_NM']             # Sido.
    for i in df_data_test:
        if i['시도'] == idx['properties']['CTP_KOR_NM']:    # Sido.json의 지역과 일치하면 pop이라는 Key를 새로 만들고
            idx['properties']['pop'] = i['사용량']         # 해당 지역의 2021.12. 시점의 물가지수를 입력
    jsondata_pick.append(idx)                               # 추가된 값을 jsondata_pick에 저장
jsondata_loc['features'] = jsondata_pick                    # jsondata_loc의 features 키에 저장

# 지도 생성, location : 시작위치, zoom_start : 시작 시 확대정도
map = folium.Map(location=[36.0068191, 127.6607805],zoom_start=7) 
cho = folium.Choropleth(geo_data=jsondata_loc,              # 색상단계도 생성, geo_data : 참고할 위치정보 데이터
                  data=df_data,                          # data : geo_data와 비교해 맞출 데이터
                  columns=['시도', '사용량'],              # columns : data에서 사용할 열
                  fill_color="YlGnBu",                      # fill_color : 색 
                  key_on='feature.id',                      # key_on : data와 맞출 geo_data의 데이터
                  legend_name='맵').add_to(map)    # legend_name : 범례 제목 입력       
cho.geojson.add_child(                                      # 속성 부여
    folium.features.GeoJsonPopup(['CTP_KOR_NM','pop'],labels=False,style='font-weight : bold')
)   # popup 넣기, geo_data의 properties 키의 값들 중 선택 가능, labels : 열이름 유무, style : 서식
# 전력량을 표시하기 위한 CircleMarker 생성
for _, row in self.df_data.iterrows():
    folium.CircleMarker(location=[row['위도'], row['경도']],
                        radius=row['전력량'], # 전력량 값을 radius로 사용
                        color='gray',
                        fill_color='gray',
                        fill_opacity=0.7,
                        tooltip=row['지역']).add_to(self.pop_map)
map

NameError: name 'self' is not defined